# Interactive 3D plot of DSO targets
Copyright Kevin Males 2022

This notebook requires Pandas, plotly and astropy. The easiest way to ensure they are all available is to use the Anaconda Python distribution.

In [12]:
import pandas as pd
import math

In [54]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import math
import numpy as np

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [78]:
df = pd.read_csv('C:\APT_Images\Processed images\Observing & Processing information.csv')

In [81]:
# use _x,_y,_z as output coords to avoid clash with typical redshift label of z

def xyz(row, ra = 'RA', dec = 'Dec', dist = 'Dist (ly)', dist_unit = u.lyr, coord_unit = None):
    ra = row[ra]
    dec = row[dec]
    if coord_unit:
        ra = ra * coord_unit
        dec = dec * coord_unit
    c = SkyCoord(ra, dec, distance = float(row[dist]) * dist_unit)
    return pd.Series([c.cartesian.x.value, c.cartesian.y.value, c.cartesian.z.value], index = ['_x', '_y', '_z'])

def xyz_log(row, ra = 'RA', dec = 'Dec', dist = 'Dist (ly)', dist_unit = u.lyr, coord_unit = None):
    ra = row[ra]
    dec = row[dec]
    if coord_unit:
        ra = ra * coord_unit
        dec = dec * coord_unit
    c = SkyCoord(ra, dec, distance = math.log10(float(row[dist])) * dist_unit)
    return pd.Series([c.cartesian.x.value, c.cartesian.y.value, c.cartesian.z.value], index = ['_x', '_y', '_z'])

In [62]:
df

,Imaging Date,Start Time,Target,Type,Caldwell,RA,Dec,Dist (ly),ISO,Lights,Darks,Flats,Filename,Processing Date,Processing notes,Notes
0,22/10/2020,21:09,Altair,Star,NaN,19h50m46.99855s,+08d52m05.9563s,16.7,1600,1 x 20s,NaN,NaN,Altair 2020-10-22,11/12/2020,"Darktable, save jpg",First session with EOS 600D attached to telesc...
1,22/10/2020,21:06,Deneb,Star,NaN,20h41m26s,+45d16m49s,2615.0,1600,1 x 20s,NaN,NaN,Deneb 2020-10-22,11/12/2020,"Darktable, save jpg",- No guiding
2,22/10/2020,21:00,Vega,Star,NaN,18h36m56.33635s,+38d47m01.2802s,25.0,800,1 x 20s,NaN,NaN,Vega 2020-10-22,6/12/2020,"Darktable, save jpg",- Experimenting with different ISO
3,5/11/2020,19:49,Albireo,Star,NaN,19h30m43.286s,+27d57m34.84s,430.0,800,1 x 20s,NaN,NaN,Albireo 2020-11-05,5/11/2020,"Darktable, save jpg",First use of APT slewing and platesolving
4,5/11/2020,19:58,Perseus double cluster,Cluster,C14,02h20m0.0s,+57d08m0.0s,7500.0,800,30 x 60s,10 x 60s,NaN,perseus double cluster,20/11/2020,"DSS, Darktable",First stacked image
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,23/3/22,20:51,NGC2403,Galaxy,C7,07h36m51.4s,+65d36m09s,9650000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN
71,24/3/22,20:35,M64 Black Eye galaxy,Galaxy,NaN,12h56m43.696s,+21d40m57.57s,17300000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN
72,24/3/22,21:56,NGC7023 Iris nebula,Nebula,C4,21h01m35.60s,+68d10m10.0s,1300.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN
73,25/3/22,22:00,M109,Galaxy,NaN,11h57m36.0s,+53d22m28s,83500000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df = df.dropna(subset = ['Dist (ly)'])

In [82]:
coords_log = df.apply(xyz_log, axis = 1)
coords_lin = df.apply(xyz, axis = 1)

In [83]:
df_log = pd.concat([df, coords_log], axis=1)
df_lin = pd.concat([df, coords_lin], axis=1)

In [84]:
fig_lin = px.scatter_3d(df_lin, x='_x', y='_y', z='_z',
              text='Target',
                   hover_name = 'Dist (ly)',
                   color = 'Type',
                   color_discrete_map = {
                       'Star': 'yellow',
                       'Nebula': 'green',
                       'Cluster': 'blue',
                       'Galaxy': 'red'
                   })

fig_lin.add_trace(go.Scatter3d(x = [0], y = [0], z = [0],
                           mode = 'markers',
                          marker = go.scatter3d.Marker(color = 'black'),
                              name = 'Earth'))

In [85]:
fig_log = px.scatter_3d(df_log, x='_x', y='_y', z='_z',
              text='Target',
                   hover_name = 'Dist (ly)',
                   color = 'Type',
                   color_discrete_map = {
                       'Star': 'yellow',
                       'Nebula': 'green',
                       'Cluster': 'blue',
                       'Galaxy': 'red'
                   })

fig_log.add_trace(go.Scatter3d(x = [0], y = [0], z = [0],
                           mode = 'markers',
                          marker = go.scatter3d.Marker(color = 'black'),
                              name = 'Earth'))

In [ ]:
fig_lin.write_html('3d_index_plot_lin.html')
fig_log.write_html('3d_index_plot_log.html')

## Plotting sources in images identified in Glade 2.3 catalog

In [4]:
# Choose non-empty row for label

def label(row):
    for col in ['HyperLEDA', '2MASS', 'SDSS-DR12']:
        n = row[col]
        if n != '---':
            return n

In [33]:
def load_glade_target(file, target):
    df =  pd.read_csv(file, sep = '\t', skiprows = [1])
    df = df[df.Dist != ' ']
    df['Dist'] = pd.to_numeric(df['Dist'])
    df['z'] = pd.to_numeric(df['z'])
    # convert MPc distance to MLy
    df['Dist'] = 3.26156 * df['Dist']
    df = df.assign(Target = target)
    labels = df.apply(label, axis = 1)
    labels.rename('Label', inplace = True)
    df = pd.concat([df, labels], axis = 1)
    return df

In [90]:
import os

glade_dir = 'c:/Astronomy/data/Glade/'

df_g = None

for f in os.listdir(glade_dir):
    target = f[:f.find('.')]
    dfg = load_glade_target(glade_dir + f, target)
    if df_g is None:
        df_g = dfg
    else:
        df_g = pd.concat([df_g, dfg], ignore_index = True)
    

In [91]:
df_g

,_RAJ2000,_DEJ2000,_V,PGC,GWGC,HyperLEDA,2MASS,SDSS-DR12,Flag1,RAJ2000,...,Bmag,BMAG,Jmag,Hmag,Kmag,Flag2,Flag3,recno,Target,Label
0,154.603683,21.893978,VizieR,30099,NGC3193,NGC3193,10182488+2153383,---,G,154.603683,...,11.780,-20.06360,8.971,8.293,8.070,3,1,796,HCG44,NGC3193
1,154.410767,21.688284,VizieR,30059,NGC3185,NGC3185,10173858+2141178,---,G,154.410767,...,12.369,-19.29350,10.232,9.588,9.323,3,1,2055,HCG44,NGC3185
2,154.138565,22.090921,VizieR,,---,30008,10163325+2205273,---,G,154.138565,...,14.368,-21.95110,11.673,10.951,10.614,3,1,9577,HCG44,30008
3,154.449371,21.873281,VizieR,30068,NGC3187,NGC3187,10174784+2152238,---,G,154.449371,...,13.295,-18.74390,12.151,11.450,11.214,3,1,20554,HCG44,NGC3187
4,154.756485,21.283783,VizieR,30133,PGC030133,PGC030133,10190155+2117016,---,G,154.756485,...,14.522,-18.64570,13.841,13.293,12.895,1,1,41084,HCG44,PGC030133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,191.067684,32.458442,VizieR,,---,---,---,124416.24+322730.3,Q,191.067684,...,,,,,,1,0,3137633,NGC4631,124416.24+322730.3
2327,191.137323,32.140674,VizieR,,---,---,---,124432.95+320826.4,Q,191.137323,...,,,,,,1,0,3137758,NGC4631,124432.95+320826.4
2328,191.158957,32.588857,VizieR,,---,---,---,124438.14+323519.8,Q,191.158957,...,,,,,,1,0,3137799,NGC4631,124438.14+323519.8
2329,191.246889,32.227660,VizieR,,---,---,---,124459.25+321339.5,Q,191.246889,...,,,,,,1,0,3137956,NGC4631,124459.25+321339.5


In [102]:
# The Glade catalog has luminosity distances which, for some nearby galaxies, don't appear to be the proper distance?
# For example, NGC 4631 has distance 1.3 Mpc or ~4 Mly, whereas it is really 30 Mly away.
# No idea why but as a workaround calculate co-moving radial distance directly from redshift.
# Not perfect for close by galaxies, but in the right vicinity.
# Code taken from James Schombert's P{ython translation of Ned Wright's cosmological distance calculator.
# https://astro.ucla.edu/~wright/CosmoCalc.html


H0 = 69.6
WM = 0.286
WV = 1 - WM

n = 1000
h = H0/100.
WR = 4.165E-5/(h*h)   # includes 3 massless neutrino species, T0 = 2.72528
WK = 1-WM-WR-WV
c = 299792.458 # velocity of light in km/sec

Tyr = 977.8    # coefficent for converting 1/H into Gyr

def dist_from_z(z):
    az = 1.0/(1+1.0*z)
    age = 0.0
    for i in range(n):
        a = az*(i+0.5)/n
        adot = math.sqrt(WK+(WM/a)+(WR/(a*a))+(WV*a*a))
        age = age + 1./adot

    zage = az*age/n
    zage_Gyr = (Tyr/H0)*zage
    DTT = 0.0
    DCMR = 0.0
    
    # do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
    for i in range(n):
        a = az+(1-az)*(i+0.5)/n
        adot = math.sqrt(WK+(WM/a)+(WR/(a*a))+(WV*a*a))
        DTT = DTT + 1./adot
        DCMR = DCMR + 1./(a*adot)

    DTT = (1.-az)*DTT/n
    DCMR = (1.-az)*DCMR/n
    age = DTT+zage
    age_Gyr = age*(Tyr/H0)
    DTT_Gyr = (Tyr/H0)*DTT
    DCMR_Gyr = (Tyr/H0)*DCMR
    DCMR_Mpc = (c/H0)*DCMR
    # return DCMR_Gyr, DCMR_Mpc, DTT_Gyr
    # return MLyr distance
    return DCMR_Gyr * 1000

In [103]:
dist_from_z(7)

28761.72821567572

In [104]:


df_g['Dist'] = df_g['z'].apply(dist_from_z)

In [105]:
# distance unit are Mlyr

coords_log = df_g.apply(xyz_log, axis = 1, args = ('_RAJ2000', '_DEJ2000', 'Dist', u.Mlyr, u.deg))
coords_lin = df_g.apply(xyz, axis = 1, args = ('_RAJ2000', '_DEJ2000', 'Dist', u.Mlyr, u.deg))

In [106]:
df_g_log = pd.concat([df_g, coords_log], axis=1)
df_g_lin = pd.concat([df_g, coords_lin], axis=1)

In [131]:
# load dataframe of named cosmological voids

df_v = pd.read_csv('voids.csv')

In [139]:
df_v

,name,ra,dec,dist
0,Local Void,18h38m00s,+18d0m0s,116
1,Giant Void,13h01m00s,+38d42m00s,1588
2,Bootes Void,14h20m00s,+26d00m00s,489
3,Eridanus Supervoid,03h15m05s,-19d35m02s,10928
4,Hercules Void,15h30m00s,+30d00m00s,326
5,Leo Void,11h30m00s,+0d00m00s,187
6,Ophiucus Void,17h00m00s,-25d00m00s,233
7,Pegasus Void,22h00m00s,+15d00m00s,257
8,Perseus-Pisces Void,1h00m00s,+10d00m00s,372


In [134]:
void_coords_log = df_v.apply(xyz_log, axis = 1, args = ('ra', 'dec', 'dist', u.Mlyr))
void_coords_lin = df_v.apply(xyz, axis = 1, args = ('ra', 'dec', 'dist', u.Mlyr))

In [136]:
df_v_log = pd.concat([df_v, void_coords_log], axis=1)
df_v_lin = pd.concat([df_v, void_coords_lin], axis=1)

In [145]:
fig_g_lin = px.scatter_3d(df_g_lin, x='_x', y='_y', z='_z',
                   hover_data = ['GWGC', 'HyperLEDA', '2MASS', 'SDSS-DR12', 'Dist', 'z'],
                          color = 'Target',
                          opacity = 0.5,
                          # symbol = 'Flag1',
                          symbol_map = {
                              'G': 'circle',  # galaxy
                              'Q': 'diamond'
                          }
                       )

fig_g_lin.add_trace(go.Scatter3d(x = [0], y = [0], z = [0],
                           mode = 'markers',
                          marker = go.scatter3d.Marker(color = 'black'),
                              name = 'Earth'))

# this will add the voids, but they're all very close to home compared to the Glade sources

#fig_g_lin.add_trace(go.Scatter3d(x = df_v_lin['_x'],
#                                 y = df_v_lin['_y'],
#                                 z = df_v_lin['_z'],
#                           mode = 'markers',
#                          marker = go.scatter3d.Marker(color = 'black', symbol = 'square'),
#                                 hovertext = df_v_lin['name'],
#                              name = 'Voids'))

In [146]:
fig_g_log = px.scatter_3d(df_g_log, x='_x', y='_y', z='_z',
              hover_data = ['GWGC', 'HyperLEDA', '2MASS', 'SDSS-DR12', 'Dist', 'z'],
                          color = 'Target',
                          opacity = 0.5,
                          # symbol = 'Flag1',
                          symbol_map = {
                              'G': 'circle',  # galaxy
                              'Q': 'diamond'
                          }
                       )


fig_g_log.add_trace(go.Scatter3d(x = [0], y = [0], z = [0],
                           mode = 'markers',
                          marker = go.scatter3d.Marker(color = 'black'),
                              name = 'Earth'))

# this will add the voids, but they're all very close to home compared to the Glade sources

#fig_g_log.add_trace(go.Scatter3d(x = df_v_log['_x'],
#                                 y = df_v_log['_y'],
#                                 z = df_v_log['_z'],
#                           mode = 'markers',
#                          marker = go.scatter3d.Marker(color = 'black', symbol = 'square'),
#                                 hovertext = df_v_log['name'],
#                              name = 'Voids'))

In [147]:
fig_g_lin.write_html('galaxy_plot_lin.html')
fig_g_log.write_html('galaxy_plot_log.html')